# Hyper-parameters tuning on HPC - NZ RSE 2020

This demo illustrates one simple way multiple ways to adapt a random search
strategy for hyper-parameters tuning to use HPC for the many parallel
computations involved.

In this example, we will rely on [Dask](https://dask.org) to do the heavy lifting,
distributing the parallel operations on SLURM jobs.

In [1]:
import json
import warnings

import numpy as np
import scipy.stats as st
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

import joblib
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from dask_ml.model_selection import HyperbandSearchCV

import torch
import skorch
from src.torch_models import SimpleCNN


Load MNIST data from [OpenML](https://www.openml.org/d/554).

In [2]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
X = X / 255.0
y = y.astype(int)

TODO display sample

To keep the example fast, let's use only a subset of the whole data set as
train and test sets.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, train_size=5000, test_size=10000, random_state=42
)

Fit a simple multi-layer perceptron neural net.

In [4]:
%%time
mlp = MLPClassifier(random_state=42).fit(X_train, y_train)

CPU times: user 26.8 s, sys: 25.3 s, total: 52.1 s
Wall time: 13.1 s


In [5]:
y_pred = mlp.predict(X_test)
mlp_acc = accuracy_score(y_test, y_pred)
print(f"Baseline MLP test accuracy is {mlp_acc * 100:.2f}%.")

Baseline MLP test accuracy is 93.77%.


Tune hyper-parameters using a random search strategy.

In [6]:
param_space = {
    "hidden_layer_sizes": st.randint(50, 200),
    "alpha": st.loguniform(1e-5, 1e-2),
    "learning_rate_init": st.loguniform(1e-4, 1e-1),
}
mlp_tuned = RandomizedSearchCV(
    MLPClassifier(random_state=42), param_space, n_iter=10, random_state=42, verbose=1
)

Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

- disabling the mechanism to write on disk when workers run out of memory,
- memory, CPUs, maximum time and number of workers per SLURM job,
- dask folders for log files and workers data.

All of these options can be set in configuration files, see [Dask configuration](https://docs.dask.org/en/latest/configuration.html)
and [Dask jobqueue configuration](https://jobqueue.dask.org/en/latest/configuration-setup.html)
for more information.

In [7]:
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10,
    processes=2,
    memory="8GiB",
    walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
    queue="bigmem",
)
client = Client(cluster)

Spawn 20 workers and connect a client to be able use them.

In [8]:
cluster.scale(n=20)
client.wait_for_workers(1)

Scikit-learn uses [Joblib](https://joblib.readthedocs.io) to parallelize
computations of many operations, including the randomized search on
hyper-parameters. If we configure Joblib to use Dask as a backend,
computations will be automatically scheduled and distributed on nodes of the
HPC platform.

In [9]:
%%time
with joblib.parallel_backend("dask", scatter=[X_train, y_train]):
    mlp_tuned.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 20 concurrent workers.


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.0min


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.0min finished


CPU times: user 14.4 s, sys: 8.09 s, total: 22.5 s
Wall time: 3min 3s


Enjoy an optimized model :).

In [10]:
y_pred_tuned = mlp_tuned.predict(X_test)
mlp_tuned_acc = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned MLP test accuracy is {mlp_tuned_acc * 100:.2f}%.")

Tuned MLP test accuracy is 95.06%.


In [11]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_tuned.best_params_, indent=4)}")

Best hyper-parameters:
{
    "alpha": 0.00010025956902289563,
    "hidden_layer_sizes": 153,
    "learning_rate_init": 0.013311216080736881
}


This first approach requires very little changes but is far from optimal from
a ressource usage perspective. The [Dask-ML](https://ml.dask.org/) package
implements a more advanced algorithm, [Hyperband](https://arxiv.org/abs/1603.06560),
designed to better use a finite bugdet, using early stopping of bad
configurations. It relies on Scikit-learn API, assuming the estimator
implements the `partial_fit` method.

In [12]:
mlp_hyper = HyperbandSearchCV(
    MLPClassifier(random_state=42),
    param_space,
    max_iter=200,
    aggressiveness=4,
    random_state=42,
)

In [13]:
%%time
_ = mlp_hyper.fit(X_train, y_train, classes=np.unique(y))

CPU times: user 6.59 s, sys: 749 ms, total: 7.34 s
Wall time: 1min 17s


In [14]:
y_pred_hyper = mlp_hyper.predict(X_test)
mlp_hyper_acc = accuracy_score(y_test, y_pred_hyper)
print(f"MLP (Hyperband) test accuracy is {mlp_hyper_acc * 100:.2f}%.")

MLP (Hyperband) test accuracy is 95.10%.


In [15]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_hyper.best_params_, indent=4)}")

Best hyper-parameters:
{
    "alpha": 2.3033838055846953e-05,
    "hidden_layer_sizes": 182,
    "learning_rate_init": 0.009713117934574873
}


Now if we want to try some deep learning models trained with GPUs, we need to
start a new Dask cluster, requesting the right resources. First let's stop the
current cluster.

In [16]:
cluster.close()
client.close()

Then start a new cluster, explicitly asking for GPUs.

In [17]:
cluster = SLURMCluster(
    cores=4,
    processes=1,
    memory="4GiB",
    walltime="0-00:30",
    log_directory="../dask/logs",
    local_directory="../dask",
    job_extra=["--gres gpu:1"],  # passed to job script to request a GPU
    queue="gpu",  # use the GPU partition
)
client = Client(cluster)

Here we use an adaptive scaling strategy, asking Dask scheduler to start at
least one worker and letting it spawning on the fly more if needed.

In [18]:
cluster.adapt(minimum=1, maximum=4)
client.wait_for_workers(1)

For the deep learning part, let's use a simple convolutional neural network
implemented using [Pytorch](https://pytorch.org/). We make use of [Skorch](https://github.com/skorch-dev/skorch)
to make it in a Scikit-learn compatible estimator.

In [19]:
def build_model(device):
    torch.manual_seed(0)
    return skorch.NeuralNetClassifier(
        module=SimpleCNN,
        module__input_dims=(28, 28),
        module__output_dim=len(np.unique(y)),
        module__n_chans=32,
        module__hidden_dim=100,
        module__dropout=0.5,
        optimizer=torch.optim.Adam,
        optimizer__lr=1e-3,
        device=device,
    )

Before training the model, we will compare timing on CPU and GPU. This will
also illustrate another aspect of the Dask API: direct submission of tasks
on the cluster.

We first dispatch training data on the cluster using `client.scatter`, to save
some bandwidth later. Here `X_train_future` is a **future** object, no yet
computed.

In [20]:
X_train_future = client.scatter(X_train.astype(np.float32))
X_train_future

<Future: finished, type: numpy.ndarray, key: ndarray-9f1886153b6feaa3490197e81e27b353>

First, let's instantiate the CPU version of the model. `client.submit` send
the computation, here `model.fit`, on a worker of the cluster and returns a
future object. This is a non-blocking call. To wait and get the result, we can
use the `result` method of the future object.

In [21]:
mlp_torch = build_model("cpu")

In [22]:
%%time
mlp_torch_future = client.submit(mlp_torch.fit, X_train_future, y_train)
mlp_torch = mlp_torch_future.result()

CPU times: user 343 ms, sys: 71.3 ms, total: 414 ms
Wall time: 22.8 s


And now let's try the GPU version.

In [23]:
mlp_torch = build_model("cuda")

In [24]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mlp_torch_future = client.submit(mlp_torch.fit, X_train_future, y_train)
    mlp_torch = mlp_torch_future.result()

CPU times: user 154 ms, sys: 33.4 ms, total: 188 ms
Wall time: 9.87 s


Thanks to Skorch, our model is compatible with Scikit-learn API, and thus can
be used with Dask-ML meta-estimators. Hence we will use Hyberband to search
for the best hyper-parameters.

In [25]:
param_space = {
    "module__n_chans": st.randint(10, 64),
    "module__hidden_dim": st.randint(50, 200),
    "module__dropout": st.uniform(),
    "optimizer__lr": st.loguniform(1e-4, 1e-1),
}
mlp_torch = HyperbandSearchCV(
    build_model("cuda"), param_space, max_iter=20, random_state=42
)

In [26]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mlp_torch.fit(X_train.astype(np.float32), y_train)

CPU times: user 2.01 s, sys: 417 ms, total: 2.42 s
Wall time: 1min 29s


In [27]:
y_pred_torch = mlp_torch.predict(X_test.astype(np.float32))
mlp_torch_acc = accuracy_score(y_test, y_pred_torch)
print(f"CNN (PyTorch & Hyperband) test accuracy is {mlp_torch_acc * 100:.2f}%.")

CNN (PyTorch & Hyperband) test accuracy is 97.45%.


In [28]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_torch.best_params_, indent=4)}")

Best hyper-parameters:
{
    "module__dropout": 0.5986584841970366,
    "module__hidden_dim": 152,
    "module__n_chans": 28,
    "optimizer__lr": 0.0001994916615063393
}
